In [1]:
# Dependancies
import pandas as pd
import os
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func
import warnings
warnings.filterwarnings('ignore')
from config import db_pass
import pymysql

In [2]:
# Establish the connection 
rds_connection_string = f"root:{db_pass}@127.0.0.1/stocks_db"
engine = create_engine(f'mysql+pymysql://{rds_connection_string}')

In [11]:
# Read all stocks for 5 years 
allstocks_df = pd.read_sql_query('select * from all_stocks_5yr', con=engine).head()
allstocks_df.head()

,id,date,open,high,low,close,volume,Name
0,1,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,3,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,4,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,5,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL


In [3]:
# Read in nasdaq 
nyse_df = pd.read_sql_query('select * from companylist_nyse', con=engine).head()
nyse_df.head(20)

,id,Symbol,Name,LastSale,MarketCap,IPOyear,Sector,industry,SummaryQuote
0,1,DDD,3D Systems Corporation,10.75,1000.0,None,Technology,Computer Software: Prepackaged Software,https://www.nasdaq.com/symbol/ddd
1,2,MMM,3M Company,190.21,200.0,None,Health Care,Medical/Dental Instruments,https://www.nasdaq.com/symbol/mmm
2,3,WBAI,500.com Limited,11.97,300.0,2013,Consumer Services,Services-Misc. Amusement & Recreation,https://www.nasdaq.com/symbol/wbai
3,4,WUBA,58.com Inc.,71.69,400.0,2013,Technology,"Computer Software: Programming, Data Processing",https://www.nasdaq.com/symbol/wuba
4,5,EGHT,8x8 Inc,23.70,500.0,None,Technology,EDP Services,https://www.nasdaq.com/symbol/eght


In [4]:
# Read in cleandf_ndaq 
nasdaq_df = pd.read_sql_query('select * from cleandf_ndaq', con=engine).head()
nasdaq_df.head()

ProgrammingError: (pymysql.err.ProgrammingError) (1146, "Table 'stocks_db.cleandf_ndaq' doesn't exist")
[SQL: select * from cleandf_ndaq]
(Background on this error at: http://sqlalche.me/e/f405)

In [ ]:
# Merging the tables

In [ ]:
# Clean the data by removing unneccesary columns for cleaner look

In [ ]:
# Run industry_analysis - count each sector

In [ ]:
# Visualize the data by sectors 